In [154]:
import pickle
import numpy
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument, LabeledSentence
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import pymongo
import nltk
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.cluster import KMeans
from nltk.corpus import stopwords
import re
from nltk.tag import pos_tag
from gensim.models import Word2Vec

from nltk.cluster import KMeansClusterer


#nltk.download('maxent_ne_chunker')
#nltk.download('words')

#nltk.download('averaged_perceptron_tagger')
#from nltk.tag import StanfordNERTagger
#from nltk.tag import StanfordPOSTagger
#import StanfordNERTaggerst = NERTagger('stanford-ner/all.3class.distsim.crf.ser.gz', 'stanford-ner/stanford-ner.jar')
#nltk.download('punkt')
#from nltk.tag.stanford import NERTagger
stop = set(stopwords.words('english'))
stemmer = SnowballStemmer("english")


In [155]:
client = pymongo.MongoClient('mongodb+srv://ridgebaseaq-docdb-dqrxk.mongodb.net/test'
                        ,username="CEDC_Quigley"
                        ,password="lpP9pCcDxQ2Y0tv8"
                        ,authSource='admin'
                        ,authMechanism='SCRAM-SHA-1')

database = client['legislative_documents']
collection = database['legiscan_people_data']
names = []
for person in collection.find():
    first = person['first_name'].lower()
    last = person['last_name'].lower()
    first_stemmed = stemmer.stem(first)
    last_stemmed = stemmer.stem(last)
    names.append(first_stemmed)
    names.append(last_stemmed)
    names.append(first)
    names.append(last)
names.append("rep")
names.append("sen")


false


In [156]:
def avg_word(sentence):
    words = sentence.split()
    return (sum(len(word) for word in words)/len(words))



In [ ]:
client = pymongo.MongoClient('mongodb+srv://ridgebaseaq-docdb-dqrxk.mongodb.net/test'
                        ,username="CEDC_Quigley"
                        ,password="lpP9pCcDxQ2Y0tv8"
                        ,authSource='admin'
                        ,authMechanism='SCRAM-SHA-1')

database = client['legislative_documents']
collection = database['legiscan_bill_metadata']
non_parsed = {}
senate_events = {}
house_events = {}
bill_events = {}
for b in collection.find({'state':'NH'}):
    bill_id = b['bill_id']
    history = b['history']
    
    non_parsed[bill_id] = []
    
    bill_events[bill_id] = []
    
    senate_events[bill_id] = []
    
    house_events[bill_id] = []
    
    for h in history:
        
        text = h['action']
        
        non_parsed[bill_id].append(text)
        
        res = remove_special_characters(text)
        
        ret = tokenize_text(res)
        
        
        
        bill_events[bill_id].append(ret)
        
        if h['chamber'] == "S":
            
            
            senate_events[bill_id].append(ret)
        elif h['chamber'] == "H":
            house_events[bill_id].append(ret)

In [ ]:
def process_events(dictionary):
    pass

for key in bill_events.keys():
    for event in bill_events[key]:
        
        res = []
        
        _words = [w for w in text.lower().split() if not w in stops]
        
        words = remove_special_characters(_words)

In [ ]:
def remove_special_characters(text):
    #table = str.maketrans('', '', string.punctuation)
    text = re.sub('[^a-zA-Z.\d\s]', '', text)
    text = text.replace('.', '')
    
    
    return text

In [ ]:
def tokenize_text(text):
    tokens = []

    for sent in nltk.sent_tokenize(text):
        for w in nltk.word_tokenize(sent):
            word = w.lower()
            if (len(word) < 2) or (not word.isalpha()) or (word in stopwords.words('english')) or (word in names):
                continue
            #tokens.append(stemmer.stem(word))              
            tokens.append(word)
    return tokens

In [ ]:
def extract_entities(text):
    for sent in nltk.sent_tokenize(text):
        for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(sent))):
            if hasattr(chunk, 'node'):
                print(chunk.node, ' '.join(c[0] for c in chunk.leaves()))

In [ ]:
#word2vec
list_of_events = []
for key in bill_events.keys():
    for e in bill_events[key]:
        list_of_events.append(e)

sentences = list_of_events

print(list_of_events[1])
model = Word2Vec(sentences, min_count=1)

NUM_CLUSTERS=40

kclusterer = KMeansClusterer(NUM_CLUSTERS, distance=nltk.cluster.util.cosine_distance, repeats=25)
assigned_clusters = kclusterer.cluster(model.wv.vocab, assign_clusters=True)
print(assigned_clusters)

In [145]:
print(list_of_events[1])

AttributeError: 'Doc2Vec' object has no attribute 'vocab'

In [ ]:

model = Doc2Vec(dm = 1, min_count=1, window=10, size=150, sample=1e-4, negative=10)
model.build_vocab(labeled_questions)

model = Doc2Vec(dm = 1, min_count=1, window=10, size=150, sample=1e-4, negative=10)
model.build_vocab(labeled_questions)

In [133]:
print(bill_events[683319])

[['introduced', 'referred', 'executive', 'departments', 'administration', 'house', 'journal', 'pg'], ['public', 'hearing', 'amendment', 'legislative', 'office', 'building'], ['executive', 'pm', 'legislative', 'office', 'building'], ['committee', 'report', 'inexpedient', 'legislate', 'mar', 'vote', 'regular', 'calendar', 'house', 'pg'], ['special', 'order', 'motion', 'failed', 'voice', 'vote', 'house', 'journal', 'pg'], ['inexpedient', 'legislate', 'motion', 'adopted', 'dv', 'house', 'journal', 'pg']]


In [126]:
print(bill_events[683319])
events = bill_events[683319]

new_events = []

for event in events:
    
    print(pos_tag(event))
    
    tockens = []
    for word in event:
        extract_entities(word)
        tockens.append(stemmer.stem(word))
    new_events.append(tockens)
#print(new_events)




[['introduced', 'referred', 'executive', 'departments', 'administration', 'house', 'journal', 'pg'], ['public', 'hearing', 'amendment', 'legislative', 'office', 'building'], ['executive', 'session', 'pm', 'legislative', 'office', 'building'], ['committee', 'report', 'inexpedient', 'legislate', 'mar', 'vote', 'regular', 'calendar', 'house', 'pg'], ['special', 'order', 'rep', 'christie', 'motion', 'failed', 'voice', 'vote', 'house', 'journal', 'pg'], ['inexpedient', 'legislate', 'motion', 'adopted', 'dv', 'house', 'journal', 'pg']]
[('introduced', 'VBN'), ('referred', 'JJ'), ('executive', 'NN'), ('departments', 'NNS'), ('administration', 'NN'), ('house', 'NN'), ('journal', 'NN'), ('pg', 'NN')]
[('public', 'JJ'), ('hearing', 'NN'), ('amendment', 'NN'), ('legislative', 'JJ'), ('office', 'NN'), ('building', 'NN')]
[('executive', 'NN'), ('session', 'NN'), ('pm', 'FW'), ('legislative', 'JJ'), ('office', 'NN'), ('building', 'NN')]
[('committee', 'NN'), ('report', 'NN'), ('inexpedient', 'NN'), 

In [ ]:
#tokenize the text
tokens = word_tokenize(billText)
#create a list of parts of speech tags for the tokens
#tags = pos_tag(tokens)
# convert to lower case
tokens = [w.lower() for w in tokens]

# remove punctuation
table = str.maketrans('', '', string.punctuation)
strippedBillText = [w.translate(table) for w in tokens]

# remove remaining tokens that are not alphabetic
wordsInBill = [word for word in strippedBillText if word.isalpha()]

#define the single character 'words' that need to be removed
singleCharacters = ['a','b','c','d','e','f','g','h','i','j',\
                    'k','l','m','n','o','p','q','r','s','t',\
                    'u','v','w','x','y','z']

#grab the list of words that were determined to be proper nouns
#ProperNouns = [word.lower() for word, tag in tags if tag == 'NNP' or tag == 'NNPS' or tag == 'IN']

#define stop words
stop_words = set(stopwords.words('english'))
# filter out stop words, single letter words, and proper nouns
wordsInBill = [w for w in wordsInBill if not w in stop_words]
wordsInBill = [w for w in wordsInBill if not w in singleCharacters]
#wordsInBill = [w for w in wordsInBill if not w in ProperNouns]

#roll words up to simpler, more categorical forms to make comparison easier
# stemming of words
#porter = PorterStemmer()
#wordsInBill = [porter.stem(word) for word in wordsInBill]

#create the corpus object
legdoc = [bill_id,doc_id,wordsInBill]
legtext_corpus.append(legdoc)



  stemmer = SnowballStemmer("english")
  for word in resume_text:
    if word not in stopwords.words('english') and not word.isdigit():
      cleaned_resume.append(word.lower())#stemmer.stem(word))

  cleaned_resume = ' '.join(cleaned_resume)